In [ ]:
# !pip install modern_robotics sympy

In [ ]:
import sympy as sp
import numpy as np
import modern_robotics as mr
from sympy import symbols, cos, sin, diff, simplify, expand, Matrix, diag, zeros, eye
from sympy.physics.mechanics import dynamicsymbols, vector, mechanics_printing

In [ ]:
mechanics_printing()

# Newton-Euler Inverse Dynamics

## Functions

In [ ]:
def so3m(v):
    return Matrix([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])

In [ ]:
def se3m(v):
    S = zeros(4)
    S[:3, :3] = vec_to_so3(v[:3])
    S[:3, 3] = v[3:]
    return S

In [ ]:
def expm3(omega, theta):
    R = eye(3) + sin(theta) * omega + \
        (1 - cos(theta)) * omega * omega
    return R


def expv3(omega, theta):
    omega = so3m(omega)
    return expm3(omega, theta)

In [ ]:
def expv6(twist, theta):
    omega = so3m(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3, :3] = expm3(omega, theta)
    T[:3, 3] = (eye(3) * theta + (1 - cos(theta)) * omega +
                (theta - sin(theta)) * omega * omega) * v
    return T

In [ ]:
def Ad(T):
    AdT = zeros(6)
    R = Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = so3m(T[:3, 3]) * R
    return AdT

In [ ]:
def ad(V):
    adV = zeros(6)
    adV[:3, :3] = so3m(V[:3])
    adV[3:, 3:] = so3m(V[:3])
    adV[3:, :3] = so3m(V[3:])
    return adV

## Define symbols

In [ ]:
th1, th2 = dynamicsymbols('theta_1, theta_2')
dth1, dth2 = dynamicsymbols('theta_1, theta_2', 1)
ddth1, ddth2 = dynamicsymbols('theta_1, theta_2', 2)
m1, m2, L1, L2, g = symbols('\mathfrak{m}_1, \mathfrak{m}_2, L_1, L_2, g')

## Define transformations, twists, and wrenches

In [ ]:
M0 = eye(4)
M1 = Matrix([[1, 0, 0, L1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
M2 = Matrix([[1, 0, 0, L1+L2], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
M3 = Matrix([[1, 0, 0, L1+L2], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

In [ ]:
M10 = M1.inv() * M0
M21 = M2.inv() * M1
M32 = M3.inv() * M2

In [ ]:
S1 = Matrix([0, 0, 1, 0, 0, 0])
S2 = Matrix([0, 0, 1, 0, -L1, 0])

In [ ]:
A1 = Ad(M1.inv()) * S1
A2 = Ad(M2.inv()) * S2

In [ ]:
G1 = diag(0, 0, 0, m1, m1, m1)
G2 = diag(0, 0, 0, m2, m2, m2)

In [ ]:
V0 = zeros(6, 1)

In [ ]:
dV0 = Matrix([0, 0, 0, 0, g, 0])

In [ ]:
F3 = zeros(6, 1)  # Wrench at the end-effector

## Forward pass

In [ ]:
T10 = expv6(-A1, th1) * M10
T21 = expv6(-A2, th2) * M21
T32 = eye(4)

In [ ]:
V1 = Ad(T10) * V0 + A1 * dth1
dV1 = Ad(T10) * dV0 + ad(V1) * A1 * dth1 + A1 * ddth1

In [ ]:
V2 = Ad(T21) * V1 + A2 * dth2
dV2 = Ad(T21) * dV1 + ad(V2) * A2 * dth2 + A2 * ddth2

## Reverse pass

In [ ]:
F2 = Ad(T32).transpose() * F3 + G2 * dV2 - ad(V2).transpose() * G2 * V2
tau2 = F2.transpose() * A2
tau2 = tau2[0].simplify()
tau2

In [ ]:
F1 = Ad(T21).transpose() * F2 + G1 * dV1 - ad(V1).transpose() * G1 * V1
tau1 = F1.transpose() * A1
tau1 = tau1[0].simplify()
tau1